# Number theory neat examples

<span style="font-size: 16pt; font-style: italic; font-weight: bold">Set 2 : </span>
<span style="font-size: 16pt; font-style: italic;">prime indexing, modular exponentiation and inverse</span>

Anton Antonov    
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
March 2025     

In [ ]:
#%js
    my $m = (12..32).pick;
    my $s = 40;
    my $batch = (($s-1) * 2 * ($s-1) / 4).round;
    my @a = ((2..$s) X (2 .. 2 * $s)).hyper(:4degree, :$batch).map({ power-mod(|$_, $m) }).rotor(2 * $s-1);
    js-d3-matrix-plot(@a, width => 600, height => 300, :!grid-lines, color-palette => 'Viridis', :!tooltip)

----

## Introduction

**What is a neat example?** : Concise or straightforward code that produces compelling visual or textual outputs.

**Maybe:** We know *neat* when we see it?

The neat examples:

- Showcase Raku programming.
- Use functionalities of different Raku modules.
- Give interesting perspectives on what is computationally possible.

Showcased:
- All number theory functions are provided by ["Math::NumberTheory"](https://raku.land/zef:antononcube/Math::NumberTheory).   
- Visualization functions are provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).
- Graphs and graph plots are provided by ["Graph"](https://raku.land/zef:antononcube/Graph).
- Data manipulation functions are provided by ["Data::Reshapers"](https://raku.land/zef:antononcube/Data::Reshapers).
- Data summarization functions are provided by ["Data::Summarizers"](https://raku.land/zef:antononcube/Data::Summarizers).
- Data translation functions (like `to-html`) are provided by ["Data::Translators"](https://raku.land/zef:antononcube/Data::Translators).

**Remark:** 
- Raku has built-in Number theory functions: `is-prime`, `mod`, `expmod`, `polymod`, `base`, `gcd`, `lcm`, etc. 
- The package "Math::NumberTheory" extends those functions and adds (many) more.

-----

## Setup

In [409]:
# Placed in in init.raku
#use Math::NumberTheory;
#use Math::NumberTheory::Utilities;

()

In [410]:
%% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [ ]:
#%js
js-d3-list-line-plot(rand xx 40, background => 'none')

In [412]:
#%js
# js-d3-list-line-plot(
#     [
#         |(100.rand xx 40).kv.map(-> $x, $y { %(:$x, :$y, group => '1') }),
#         |(100.rand xx 40).kv.map(-> $x, $y { %(:$x, :$y, group => '2') })
#     ].flat,
#     background => 'none'
# )

In [413]:
my %opts =  
    background => '#1F1F1F',
    title-color => 'Ivory';

{background => #1F1F1F, title-color => Ivory}

------

## Prime subs

### Prime indexing

There is a function `prime(n)` that gives the n-th prime number:

In [414]:
[10, 20 ... 100]».&prime

[29 71 113 173 229 281 349 409 463 541]

In [415]:
prime([12, 123, 200])

[37 677 1223]

Next primes:

In [416]:
say next-prime(33.4);
say next-prime(33, 3);

37
43


Different signatures:

In [417]:
say next-prime(33, [3, 4, 9]);
say next-prime([33, 67], 3);
say next-prime([33, 67], [3, 4, 9]);

(43 47 71)
(43 79)
((43 47 71) (79 83 107))


Here is one way to use `prime` -- to show [prime gaps](https://mathworld.wolfram.com/PrimeGaps.html):

In [ ]:
#% js
js-d3-list-line-plot( 
  (1..700).map({ [$_, prime($_ + 1) - prime($_) ] }),
    :1200width,
    :400height,
    title => 'Prime gaps',
    x-label => 'prime index',
    y-label => 'gap',
    |%opts
    )

### Different kind of primes

Computation of [twin primes](https://en.wikipedia.org/wiki/Twin_prime), [cousin primes](https://en.wikipedia.org/wiki/Cousin_prime), and [sexy primes](https://en.wikipedia.org/wiki/Sexy_prime):

In [419]:
twin-primes(12)

((3 5) (5 7) (11 13) (17 19) (29 31) (41 43) (59 61) (71 73) (101 103) (107 109) (137 139) (149 151))

In [420]:
cousin-primes(12)

((3 7) (7 11) (13 17) (19 23) (37 41) (43 47) (67 71) (79 83) (97 101) (103 107) (109 113) (127 131))

In [421]:
sexy-primes(12)

((5 11) (7 13) (11 17) (13 19) (17 23) (23 29) (31 37) (37 43) (41 47) (47 53) (53 59) (61 67))

Visualizing the twin primes via Ulam's spiral:

In [ ]:
#% js
    my @ps = twin-primes(400);
    my $np = @ps.flat(:hammer).max;
    my $fs = Set.new(@ps».head);
    my $ss = Set.new(@ps».tail);
    my @spiral = Math::SparseMatrix.new(dense-matrix => spiral-lattice($np.sqrt.ceiling)).tuples;
    @spiral .= map({ (<x y group> Z=> $_.Array).Hash });
    @spiral .= map({

        $_<group> = do given $_<group> { 
            when $_ ∈ $fs { 'first' } 
            when $_ ∈ $ss { 'second' } 
            when is-prime($_) { 'prime' }  
            default { 'number' }  
        }
        $_
    });

    my $color-scheme = 'Tableau10'; # <Set1 Set2 Dark2 Observable10>.pick;

    js-d3-list-plot(
        @spiral.grep({ $_<group> ∈ <first second> }),
        background => 'none',
        :legends,
        :!axes,
        :$color-scheme,
        :530width, :430height,
        margins => {:20top, :10bottom, :10left, :10right},
        title => "$np : $color-scheme",
        :12title-font-size,
        title-color => 'Ivory'
    )

As a reminder -- here Ulam spiral table:

In [423]:
#% html
my $n = 17;
spiral-lattice($n):d
==> { $_.deepmap({ is-prime($_) ?? $_ !! '' }) }()
==> to-html(field-names => (^$n)».Str)

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
257,,,,,,251,,,,,,,,,,241
,197,,,,193,,191,,,,,,,,,
,,,,,,,,139,,137,,,,,,239
,199,,101,,,,97,,,,,,,,181,
,,,,,,,,61,,59,,,,131,,
,,,103,,37,,,,,,31,,89,,179,
263,,149,,67,,17,,,,13,,,,,,
,,,,,,,5,,3,,29,,,,,
,,151,,,,19,,,2,11,,53,,127,,233
,,,107,,41,,7,,,,,,,,,


----

## Modular exponentiation (power mod)

The function `power-mod(a, b, m)` gives $a^b \bmod m$.

**Remark:** The function `power-mod` is an upgrade of the built-in `expmod` -- `power-mod` works on Gaussian integers. The name `power-mod` is consistent with `PowerMod` of Wolfram Language.

In [424]:
power-mod(2 + i, 2, 3)

0+1i

Plot a list of powers of 3 where the exponent is varied, modulo some prime number:

In [ ]:
#%js
my $i = [44, 83, 117, 157, 123, 129].pick;
my @a = (1 .. (prime($i)-1) ).map({ power-mod(3, $_ , prime($i)) });
    my %opts = title-color => 'DimGray',
        plot-label => "3ᵇ mod {prime($i)}",
        background => 'none',
        width => 800,
        height => 200,
        :!axes;

    js-d3-list-plot(@a, |%opts) ~ "\n" ~ js-d3-list-line-plot(@a, |%opts)

Plot values of varying powers of numbers with a fixed modulus:

In [ ]:
#%js
my @a = ((2..100) X (2..100)).map({ power-mod($_.head, $_.tail, 32) }).rotor(99);
js-d3-matrix-plot(@a, width => 400, height => 400, :!grid-lines, :!tooltip) 

Plot an a version of [Ulam spiral](https://en.wikipedia.org/wiki/Ulam_spiral) where numbers are colored based on `power-mod`:

In [ ]:
#% js
my @mat = spiral-lattice(101).deepmap({ power-mod($_, 3, 17) });
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, :!tooltip, color-palette => 'Rainbow')

----

## Modular inverse

A number $b$ is called a *modular multiplicative inverse* of $a$ modulo $n$ if $(a\,b) \bmod n = 1$.

The function `modular-inverse(k,n)` gives the modular inverse of $k$ modulo $n$.

- `modular-inverse` is also known as modular multiplicative inverse.
- Typically used in modular arithmetic and cryptography.
- `modular-inverse(k,n)` gives the number $r$ such that the remainder of the division of $r k$ by $n$ is equal to $1$.
- If $k$ and $n$ are not coprime, no modular inverse exists and `modular-inverse(k,n)` returns `Nil`.

In [428]:
#%html
my @row = 2, 3, 4, 5, 7;
my @col = 1..10;
(@row X @col).map({ $_.tail => modular-inverse($_.tail, $_.head) // '' }).rotor(@col.elems)».Hash
==> { $_.kv.map(-> $k, %v { %v<n> = @row[$k]; %v }) }()
==> to-html(field-names => ['n', |@col».Str])

n,1,2,3,4,5,6,7,8,9,10
2,1,,1,,1,,1,,1,
3,1,2,,1,2,,1,2,,1
4,1,,3,,1,,3,,1,
5,1,3,2,4,,1,3,2,4,
7,1,4,5,2,3,6,,1,4,5


Here is a table exemplifying the relationship between modular inverse and modular exponentiation:

<table border="1"><thead><tr><th>code</th><th>result</th></tr></thead><tbody><tr><td align=left>power-mod(9, -1, 5)</td><td align=left>4</td></tr><tr><td align=left>modular-inverse(9, 5)</td><td align=left>4</td></tr><tr><td align=left>4 * 9 mod 5</td><td align=left>1</td></tr></tbody></table>

Visualize when a number is invertible a random modulus:

In [ ]:
#% js
my $n = 21;
my $m = [12, 14, 20].pick;
my @a = ((1 .. $n) X (1 .. 3 * $n)).map({ not so modular-inverse($_.head + $_.tail ** 2, $m) })».Int.rotor(3 * $n);

js-d3-matrix-plot(@a, width => 900, height => 300, :!grid-lines, color-palette => 'Blues', :!tooltip)

----

## Sunflower embeddings

"Math::NumberTheory" provides a spiral embeddings sub: 

In [ ]:
#% js
my $np = 2_500;
[21, 32, 68].map( -> $p {
        my $color-scheme = <Set1 Set2 Dark2 Observable10>.pick;

        js-d3-list-plot( 
            sunflower-embedding(
                $np,
                with => { (modular-inverse($_, $p) ?? min(modular-inverse($_, $p), $_ mod $p) !! 0).Str }, 
                :dataset),
            background => 'none',
            :!legends,
            :!axes,
            :$color-scheme,
            :320width, :330height,
            margins => {:20top, :10bottom, :10left, :10right},
            title => "$p : $color-scheme",
            :12title-font-size,
            title-color => 'Ivory'
        )
    }).join("\n")

The embedding plots use the modulus reminders for color if the modulus inverse exists. 

Using:

- Small numbers (e.g. `2000`) produces informative results
- Moderately large numbers (e.g. `6000`) produces smoother plots
- Large numbers (`> 12000`) produces plots that get "smeared"

----

## Chords visualization

- Circle chord visualizations fit well modular arithmetic properties.
- Graphs and graphs plots can be leveraged.
- Those plots are not necessarily informative.
- Modular inverse chord plots are _not_ pretty for small moduli.
    - Much better with larger ones.


Here is chords-plot function using Graphviz via "Graph":

In [431]:
sub mod-inverse-chords-plot( Int:D $n, *%args) {
    my @edges = (1..$n).map({ modular-inverse($_, $n) && modular-inverse($_, $n) != $_ ?? Pair.new($_.Str, modular-inverse($_, $n).Str) !! Empty  });
    my $vertex-coordinates = Graph::Wheel($n-1).vertex-coordinates; 
    $vertex-coordinates<0>:delete;
    # rotate by π/2
    $vertex-coordinates = $vertex-coordinates.map({ $_.key => (-$_.value.tail, $_.value.head )}).Hash;
    my $g = Graph.new(@edges, :!directed-edges, :$vertex-coordinates);

    my %default-args =
        background => 'none', 
        node-shape => 'point', 
        node-width => 0.02, 
        graph-size => 3,
        edge-thickness => 0.7, 
        graph-label => $n.Str, 
        font-color => 'Ivory', 
        :!node-labels, 
        :8font-size, 
        vertex-color => '#1F1F1F',
        engine => 'neato'
        ;

    my %args2 = %default-args , %args;

    $g.dot(|%args2):svg
}

&mod-inverse-chords-plot

Using typical graph edges:

In [432]:
#% html
(260..264).map({ mod-inverse-chords-plot($_, splines => 'line', :2graph-size) }).join("\n")

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
 260 
<!-- 196 -->
 
 196 
 
 
<!-- 230 -->
 
 230 
 
 
<!-- 24 -->
 
 24 
 
 
<!-- 122 -->
 
 122 
 
 
<!-- 172 -->
 
 172 
 
 
<!-- 135 -->
 
 135 
 
 
<!-- 252 -->
 
 252 
 
 
<!-- 15 -->
 
 15 
 
 
<!-- 171 -->
 
 171 
 
 
<!-- 143 -->
 
 143 
 
 
<!-- 110 -->
 
 110 
 
 
<!-- 11 -->
 
 11 
 
 
<!-- 71 -->
 
 71 
 
 
<!-- 11--71 -->
 
 11--71 
 
 
<!-- 107 -->
 
 107 
 
 
<!-- 243 -->
 
 243 
 
 
<!-- 107--243 -->
 
 107--243 
 
 
<!-- 227 -->
 
 227 
 
 
<!-- 63 -->
 
 63 
 
 
<!-- 227--63 -->
 
 227--63 
 
 
<!-- 168 -->
 
 168 
 
 
<!-- 249 -->
 
 249 
 
 
<!-- 57 -->
 
 57 
 
 
<!-- 73 -->
 
 73 
 
 
<!-- 57--73 -->
 
 57--73 
 
 
<!-- 117 -->
 
 117 
 
 
<!-- 1 -->
 
 1 
 
 
<!-- 134 -->
 
 134 
 
 
<!-- 67 -->
 
 67 
 
 
<!-- 140 -->
 
 140 
 
 
<!-- 181 -->
 
 181 
 
 
<!-- 68 -->
 
 68 
 
 
<!-- 160 -->
 
 160 
 
 
<!-- 85 -->
 
 85 
 
 
<!-- 190 -->
 
 190 
 
 
<!-- 187 -->
 
 187 
 
 
<!-- 203 -->
 
 203 
 
 
<!-- 187--203 -->
 
 187--203 
 
 
<!-- 235 -->
 
 235 
 
 
<!-- 10 -->
 
 10 
 
 
<!-- 184 -->
 
 184 
 
 
<!-- 36 -->
 
 36 
 
 
<!-- 76 -->
 
 76 
 
 
<!-- 178 -->
 
 178 
 
 
<!-- 114 -->
 
 114 
 
 
<!-- 100 -->
 
 100 
 
 
<!-- 52 -->
 
 52 
 
 
<!-- 38 -->
 
 38 
 
 
<!-- 98 -->
 
 98 
 
 
<!-- 153 -->
 
 153 
 
 
<!-- 17 -->
 
 17 
 
 
<!-- 153--17 -->
 
 153--17 
 
 
<!-- 166 -->
 
 166 
 
 
<!-- 59 -->
 
 59 
 
 
<!-- 221 -->
 
 221 
 
 
<!-- 177 -->
 
 177 
 
 
<!-- 213 -->
 
 213 
 
 
<!-- 177--213 -->
 
 177--213 
 
 
<!-- 251 -->
 
 251 
 
 
<!-- 226 -->
 
 226 
 
 
<!-- 241 -->
 
 241 
 
 
<!-- 41 -->
 
 41 
 
 
<!-- 241--41 -->
 
 241--41 
 
 
<!-- 102 -->
 
 102 
 
 
<!-- 225 -->
 
 225 
 
 
<!-- 159 -->
 
 159 
 
 
<!-- 90 -->
 
 90 
 
 
<!-- 194 -->
 
 194 
 
 
<!-- 34 -->
 
 34 
 
 
<!-- 229 -->
 
 229 
 
 
<!-- 42 -->
 
 42 
 
 
<!-- 175 -->
 
 175 
 
 
<!-- 116 -->
 
 116 
 
 
<!-- 89 -->
 
 89 
 
 
<!-- 199 -->
 
 199 
 
 
<!-- 237 -->
 
 237 
 
 
<!-- 32 -->
 
 32 
 
 
<!-- 158 -->
 
 158 
 
 
<!-- 157 -->
 
 157 
 
 
<!-- 53 -->
 
 53 
 
 
<!-- 157--53 -->
 
 157--53 
 
 
<!-- 137 -->
 
 137 
 
 
<!-- 93 -->
 
 93 
 
 
<!-- 137--93 -->
 
 137--93 
 
 
<!-- 254 -->
 
 254 
 
 
<!-- 56 -->
 
 56 
 
 
<!-- 82 -->
 
 82 
 
 
<!-- 84 -->
 
 84 
 
 
<!-- 193 -->
 
 193 
 
 
<!-- 97 -->
 
 97 
 
 
<!-- 193--97 -->
 
 193--97 
 
 
<!-- 62 -->
 
 62 
 
 
<!-- 223 -->
 
 223 
 
 
<!-- 7 -->
 
 7 
 
 
<!-- 223--7 -->
 
 223--7 
 
 
<!-- 236 -->
 
 236 
 
 
<!-- 16 -->
 
 16 
 
 
<!-- 136 -->
 
 136 
 
 
<!-- 192 -->
 
 192 
 
 
<!-- 239 -->
 
 239 
 
 
<!-- 99 -->
 
 99 
 
 
<!-- 239--99 -->
 
 239--99 
 
 
<!-- 240 -->
 
 240 
 
 
<!-- 58 -->
 
 58 
 
 
<!-- 6 -->
 
 6 
 
 
<!-- 13 -->
 
 13 
 
 
<!-- 123 -->
 
 123 
 
 
<!-- 167 -->
 
 167 
 
 
<!-- 123--167 -->
 
 123--167 
 
 
<!-- 128 -->
 
 128 
 
 
<!-- 197 -->
 
 197 
 
 
<!-- 33 -->
 
 33 
 
 
<!-- 197--33 -->
 
 197--33 
 
 
<!-- 92 -->
 
 92 
 
 
<!-- 119 -->
 
 119 
 
 
<!-- 119--59 -->
 
 119--59 
 
 
<!-- 244 -->
 
 244 
 
 
<!-- 18 -->
 
 18 
 
 
<!-- 200 -->
 
 200 
 
 
<!-- 165 -->
 
 165 
 
 
<!-- 55 -->
 
 55 
 
 
<!-- 209 -->
 
 209 
 
 
<!-- 206 -->
 
 206 
 
 
<!-- 205 -->
 
 205 
 
 
<!-- 60 -->
 
 60 
 
 
<!-- 113 -->
 
 113 
 
 
<!-- 113--237 -->
 
 113--237 
 
 
<!-- 250 -->
 
 250 
 
 
<!-- 183 -->
 
 183 
 
 
<!-- 27 -->
 
 27 
 
 
<!-- 183--27 -->
 
 183--27 
 
 
<!-- 179 -->
 
 179 
 
 
<!-- 179--199 -->
 
 179--199 
 
 
<!-- 26 -->
 
 26 
 
 
<!-- 218 -->
 
 218 
 
 
<!-- 66 -->
 
 66 
 
 
<!-- 69 -->
 
 69 
 
 
<!-- 14 -->
 
 14 
 
 
<!-- 224 -->
 
 224 
 
 
<!-- 95 -->
 
 95 
 
 
<!-- 222 -->
 
 222 
 
 
<!-- 228 -->
 
 228 
 
 
<!-- 185 -->
 
 185 
 
 
<!-- 233 -->
 
 233 
 
 
<!-- 77 -->
 
 77 
 
 
<!-- 233--77 -->
 
 233--77 
 
 
<!-- 46 -->
 
 46 
 
 
<!-- 245 -->
 
 245

Using orthogonal lines for the graph edges:

In [470]:
#% html
(130...132).map({ mod-inverse-chords-plot($_, splines => 'ortho', :1edge-thickness, :3graph-size) }).join("\n")

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
 130 
<!-- 16 -->
 
 16 
 
 
<!-- 116 -->
 
 116 
 
 
<!-- 104 -->
 
 104 
 
 
<!-- 23 -->
 
 23 
 
 
<!-- 39 -->
 
 39 
 
 
<!-- 52 -->
 
 52 
 
 
<!-- 18 -->
 
 18 
 
 
<!-- 88 -->
 
 88 
 
 
<!-- 20 -->
 
 20 
 
 
<!-- 81 -->
 
 81 
 
 
<!-- 64 -->
 
 64 
 
 
<!-- 57 -->
 
 57 
 
 
<!-- 73 -->
 
 73 
 
 
<!-- 57--73 -->
 
 57--73 
 
 
<!-- 48 -->
 
 48 
 
 
<!-- 82 -->
 
 82 
 
 
<!-- 118 -->
 
 118 
 
 
<!-- 21 -->
 
 21 
 
 
<!-- 31 -->
 
 31 
 
 
<!-- 21--31 -->
 
 21--31 
 
 
<!-- 55 -->
 
 55 
 
 
<!-- 30 -->
 
 30 
 
 
<!-- 43 -->
 
 43 
 
 
<!-- 3 -->
 
 3 
 
 
<!-- 87 -->
 
 87 
 
 
<!-- 3--87 -->
 
 3--87 
 
 
<!-- 76 -->
 
 76 
 
 
<!-- 4 -->
 
 4 
 
 
<!-- 93 -->
 
 93 
 
 
<!-- 70 -->
 
 70 
 
 
<!-- 71 -->
 
 71 
 
 
<!-- 51 -->
 
 51 
 
 
<!-- 101 -->
 
 101 
 
 
<!-- 121 -->
 
 121 
 
 
<!-- 101--121 -->
 
 101--121 
 
 
<!-- 125 -->
 
 125 
 
 
<!-- 117 -->
 
 117 
 
 
<!-- 92 -->
 
 92 
 
 
<!-- 7 -->
 
 7 
 
 
<!-- 7--93 -->
 
 7--93 
 
 
<!-- 111 -->
 
 111 
 
 
<!-- 41 -->
 
 41 
 
 
<!-- 111--41 -->
 
 111--41 
 
 
<!-- 83 -->
 
 83 
 
 
<!-- 34 -->
 
 34 
 
 
<!-- 109 -->
 
 109 
 
 
<!-- 99 -->
 
 99 
 
 
<!-- 109--99 -->
 
 109--99 
 
 
<!-- 15 -->
 
 15 
 
 
<!-- 46 -->
 
 46 
 
 
<!-- 97 -->
 
 97 
 
 
<!-- 5 -->
 
 5 
 
 
<!-- 122 -->
 
 122 
 
 
<!-- 33 -->
 
 33 
 
 
<!-- 67 -->
 
 67 
 
 
<!-- 33--67 -->
 
 33--67 
 
 
<!-- 49 -->
 
 49 
 
 
<!-- 69 -->
 
 69 
 
 
<!-- 49--69 -->
 
 49--69 
 
 
<!-- 13 -->
 
 13 
 
 
<!-- 124 -->
 
 124 
 
 
<!-- 79 -->
 
 79 
 
 
<!-- 126 -->
 
 126 
 
 
<!-- 56 -->
 
 56 
 
 
<!-- 14 -->
 
 14 
 
 
<!-- 65 -->
 
 65 
 
 
<!-- 44 -->
 
 44 
 
 
<!-- 86 -->
 
 86 
 
 
<!-- 10 -->
 
 10 
 
 
<!-- 106 -->
 
 106 
 
 
<!-- 98 -->
 
 98 
 
 
<!-- 42 -->
 
 42 
 
 
<!-- 95 -->
 
 95 
 
 
<!-- 66 -->
 
 66 
 
 
<!-- 60 -->
 
 60 
 
 
<!-- 22 -->
 
 22 
 
 
<!-- 25 -->
 
 25 
 
 
<!-- 74 -->
 
 74 
 
 
<!-- 50 -->
 
 50 
 
 
<!-- 45 -->
 
 45 
 
 
<!-- 90 -->
 
 90 
 
 
<!-- 9 -->
 
 9 
 
 
<!-- 1 -->
 
 1 
 
 
<!-- 129 -->
 
 129 
 
 
<!-- 58 -->
 
 58 
 
 
<!-- 54 -->
 
 54 
 
 
<!-- 123 -->
 
 123 
 
 
<!-- 37 -->
 
 37 
 
 
<!-- 123--37 -->
 
 123--37 
 
 
<!-- 100 -->
 
 100 
 
 
<!-- 85 -->
 
 85 
 
 
<!-- 63 -->
 
 63 
 
 
<!-- 63--97 -->
 
 63--97 
 
 
<!-- 107 -->
 
 107 
 
 
<!-- 113 -->
 
 113 
 
 
<!-- 107--113 -->
 
 107--113 
 
 
<!-- 68 -->
 
 68 
 
 
<!-- 32 -->
 
 32 
 
 
<!-- 38 -->
 
 38 
 
 
<!-- 12 -->
 
 12 
 
 
<!-- 11 -->
 
 11 
 
 
<!-- 11--71 -->
 
 11--71 
 
 
<!-- 62 -->
 
 62 
 
 
<!-- 110 -->
 
 110 
 
 
<!-- 112 -->
 
 112 
 
 
<!-- 102 -->
 
 102 
 
 
<!-- 53 -->
 
 53 
 
 
<!-- 24 -->
 
 24 
 
 
<!-- 47 -->
 
 47 
 
 
<!-- 47--83 -->
 
 47--83 
 
 
<!-- 120 -->
 
 120 
 
 
<!-- 115 -->
 
 115 
 
 
<!-- 114 -->
 
 114 
 
 
<!-- 36 -->
 
 36 
 
 
<!-- 80 -->
 
 80 
 
 
<!-- 94 -->
 
 94 
 
 
<!-- 108 -->
 
 108 
 
 
<!-- 119 -->
 
 119 
 
 
<!-- 59 -->
 
 59 
 
 
<!-- 119--59 -->
 
 119--59 
 
 
<!-- 77 -->
 
 77 
 
 
<!-- 26 -->
 
 26 
 
 
<!-- 35 -->
 
 35 
 
 
<!-- 91 -->
 
 91 
 
 
<!-- 61 -->
 
 61 
 
 
<!-- 61--81 -->
 
 61--81 
 
 
<!-- 127 -->
 
 127 
 
 
<!-- 127--43 -->
 
 127--43 
 
 
<!-- 78 -->
 
 78 
 
 
<!-- 103 -->
 
 103 
 
 
<!-- 103--77 -->
 
 103--77 
 
 
<!-- 105 -->
 
 105 
 
 
<!-- 8 -->
 
 8 
 
 
<!-- 17 -->
 
 17 
 
 
<!-- 17--23 -->
 
 17--23 
 
 
<!-- 75 -->
 
 75 
 
 
<!-- 96 -->
 
 96 
 
 
<!-- 19 -->
 
 19 
 
 
<!-- 89 -->
 
 89 
 
 
<!-- 19--89 -->
 
 19--89 
 
 
<!-- 72 -->
 
 72 
 
 
<!-- 40 -->
 
 40 
 
 
<!-- 84 -->
 
 84 
 
 
<!-- 27 -->
 
 27 
 
 
<!-- 27--53 -->
 
 27--53 
 
 
<!-- 29 -->
 
 29 
 
 
<!-- 29--9 -->
 
 29--9 
 
 
<!-- 128 -->
 
 128 
 
 
<!-- 2 -->
 
 2 
 
 
<!-- 28 -->
 
 28 
 
 
<!-- 6 -->
 
 6 
 
 
 
 

<?xml version="1.0" encoding="UTF-8" stand

Using large(r) numbers:

In [ ]:
#% html
(3000, 3005, 5015).map({ mod-inverse-chords-plot($_, splines => 'line', edge-thickness => 0.08, :4graph-size) }).join("\n")

Using D3.js:

In [ ]:
#% js
(2_000, 4_000 ... 16_000).map( -> $n { 
    my &f = -> $x { modular-inverse($x, $n) && modular-inverse($x, $n) != $x ?? Pair.new($x, modular-inverse($x, $n)) !! Empty };
    
    my @segments = circular-chords-tracing($n, with => &f, :d);
    
    @segments .= map({ $_<group> = $_<group>.Str; $_ });
    
    js-d3-list-line-plot(
        @segments,
        stroke-width => $n < 8000 ?? 0.08 !! 0.04,
        background => '#1F1F1F',
        :250width, :250height,
        :!axes,
        :!legends,
        :10margins,
        color-scheme => 'Ivory'
        )
}).join("\n")